<a href="https://colab.research.google.com/github/BenjamintsKang/AIFFEL_Quest/blob/master/01_Base_RAG_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 공통 작업

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("HIH-Base-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
HIH-Base-RAG


# PDF 파일을 사용하는 경우

In [ ]:
# 파일 포맷에 따라 Loader를 선택해 import하기
# from langchain.document_loaders import TextLoader, CSVLoader, PyPDFLoader
# from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
# from langchain_community.document_loaders import PDFMinerLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# Vector Store 종류는 엄청 다양함. 이번에는 Chroma 대신 FAISS 사용
# from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

FILE_PATH = "./data/ssen_single_test.pdf"

# 1. 데이터 로드
loader = PDFPlumberLoader(FILE_PATH)
docs = loader.load()

In [ ]:
docs

[Document(metadata={'source': './data/ssen_single_test.pdf', 'file_path': './data/ssen_single_test.pdf', 'page': 0, 'total_pages': 8, 'CreationDate': "D:20240908161921+09'00'", 'Creator': 'PDF 2022 12.0.0.3970', 'ModDate': "D:20240908162013+09'00'", 'Producer': 'PDF 2022 12.0.0.3970'}, page_content='건국대학교 (05029) 서울특별시 광진구 능동로 120\n서\n입학처: (02) 450-0007\n울\n권\n1 2024 대입과 달라진 점\n주요 변경 전년도와 달라진 점\n전형유형 전형명\n내용 2025학년도 2024학년도\n∙ KU자유전공학부(248명 선발)\n∙ 문과대학자유전공학부(49명 선발)\n∙ 이과대학자유전공학부(24명 선발)\n전공자율선택제 모집단위 신설 ∙ 공과대학자유전공학부(135명 선발)\n∙ 사회과학대학융합전공학부(75명 선발)\n∙ 융합과학기술원자유전공학부(33명 선발)\n∙ 생명과학대학자유전공학부(46명 선발)\n학생부교과 KU지역균형 지원자격 ∙ 지원불가 대상에 각종학교 졸업(예정)자 추가\n∙ 국가보훈대상자, 의사상자 및 자녀,\n학생부종합 사회통합 지원자격 ∙ 국가보훈대상자, 농어촌학생, 기초생활및차상위 직업군인, 소방공무원, 경찰공무원 자녀,\n다자녀(4자녀 이상) 가정 등\n∙ 인문: 국,수,영,탐(1) 중 2개 합 5,\n한국사 5 이내\n∙ 인문/자연/KU자유전공학부: 국,수,영,탐(1) 중 2개\nKU논술 수능최저학력 ∙ 자연: 국,수(미/기),영,과(1) 중 2개 합 5,\n논술 합 5, 한국사 5 이내\n우수자 기준 반영과목 한국사 5 이내\n∙ 수의예과: 국,수,영,탐(1) 중 3개 합 4, 한국사 5 이내\n∙ 수의예과: 국,수(미/기),영,과(1) 중 3개 합\n4, 한국사 5 이내

In [ ]:
docs[0]

Document(metadata={'source': './data/ssen_single_test.pdf', 'file_path': './data/ssen_single_test.pdf', 'page': 0, 'total_pages': 8, 'CreationDate': "D:20240908161921+09'00'", 'Creator': 'PDF 2022 12.0.0.3970', 'ModDate': "D:20240908162013+09'00'", 'Producer': 'PDF 2022 12.0.0.3970'}, page_content='건국대학교 (05029) 서울특별시 광진구 능동로 120\n서\n입학처: (02) 450-0007\n울\n권\n1 2024 대입과 달라진 점\n주요 변경 전년도와 달라진 점\n전형유형 전형명\n내용 2025학년도 2024학년도\n∙ KU자유전공학부(248명 선발)\n∙ 문과대학자유전공학부(49명 선발)\n∙ 이과대학자유전공학부(24명 선발)\n전공자율선택제 모집단위 신설 ∙ 공과대학자유전공학부(135명 선발)\n∙ 사회과학대학융합전공학부(75명 선발)\n∙ 융합과학기술원자유전공학부(33명 선발)\n∙ 생명과학대학자유전공학부(46명 선발)\n학생부교과 KU지역균형 지원자격 ∙ 지원불가 대상에 각종학교 졸업(예정)자 추가\n∙ 국가보훈대상자, 의사상자 및 자녀,\n학생부종합 사회통합 지원자격 ∙ 국가보훈대상자, 농어촌학생, 기초생활및차상위 직업군인, 소방공무원, 경찰공무원 자녀,\n다자녀(4자녀 이상) 가정 등\n∙ 인문: 국,수,영,탐(1) 중 2개 합 5,\n한국사 5 이내\n∙ 인문/자연/KU자유전공학부: 국,수,영,탐(1) 중 2개\nKU논술 수능최저학력 ∙ 자연: 국,수(미/기),영,과(1) 중 2개 합 5,\n논술 합 5, 한국사 5 이내\n우수자 기준 반영과목 한국사 5 이내\n∙ 수의예과: 국,수,영,탐(1) 중 3개 합 4, 한국사 5 이내\n∙ 수의예과: 국,수(미/기),영,과(1) 중 3개 합\n4, 한국사 5 이내\

In [ ]:
# 2. OpenAI의 Tokenizer인 tiktoken 사용해 분할하기
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    # chunk_size=30000,
    # chunk_overlap=6000,
    chunk_size=800,
    chunk_overlap=200,
)
splitted_docs = splitter.split_documents(docs)

# 3. OpenAI 임베딩 객체 생성
embeddings = OpenAIEmbeddings()
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")   # 최신 임베딩 모델로 지정! (저렴하고 성능도 우수함) --> 모델을 못 찾음. ㅠ

# 4. Vector Store 생성 및 캐싱
# [주의] OpenAIEmbeddings() 객체인 embeddings를 사용해서 분할한 문서에 대한 vectorization을 진행하기 때문에 Embedding 모델 사용료가 나감!! (따라서, 여러번 실행하지 말 것!!)
cache_dir = LocalFileStore("./.cache/jinhak/")
# 첫번째 인자로 캐싱할 OpenAIEmbeddings 객체를 전달
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 캐싱 디렉토리에 캐싱된 임베딩이 있으면 그것을 사용해서 Vector Store 생성
# 만야, 맨 처음 실행되는 상황이라 캐싱된 임베딩이 없으면 cache_embeddings 생성할 때 인자로 넘겨준 최초의 임베딩을 사용함.
# vectorstore = Chroma.from_documents(splitted_docs4, cache_embeddings)
# 실제로 해 보면 Chroma에 비해 FAISS를 사용했을 때 더 좋은 성능을 보이는 경우가 많다고 함.
vectorstore = FAISS.from_documents(splitted_docs, cache_embeddings)

In [ ]:
# [테스트] Vector Store에서 인자로 넘긴 문장과 유사도가 높은 청크 문서 찾기
result_docs = vectorstore.similarity_search("전형 종류에 대해 알려줘.")
# result_docs = vectorstore.similarity_search("전형 종류에 대해 알려줘.", k=6)  # 반환되는 문서 수를 6개로 설정(기본값: 4개)
# result_docs = vectorstore.similarity_search("서강대학교 입시 결과 알려줘.")
result_docs

[Document(metadata={'source': './data/ssen_single_test.pdf', 'file_path': './data/ssen_single_test.pdf', 'page': 4, 'total_pages': 8, 'CreationDate': "D:20240908161921+09'00'", 'Creator': 'PDF 2022 12.0.0.3970', 'ModDate': "D:20240908162013+09'00'", 'Producer': 'PDF 2022 12.0.0.3970'}, page_content='∙ 특히 올해 신설되는 KU자유전공학부는 학업적 기초역량과 다양한 경험을 바탕으로 융·복합적 소양을 기른 학생, 깊이있는 탐색으로\n학문간 응용능력을 갖춘 학생을 인재상으로 설정하고 있음. 구체적으로 다양한 활동을 탐색하여 관심의 다양성이 나타나는 탐색형,\n관심있는 것에 집중하여 관심의 깊이가 나타나는 집중형으로 설정하고 있음. 기존의 서류평가 요소 중 진로역량 대신 현재의 상황이나\n수준보다 질적으로 더 높은 단계로 향상시킬 수 있는 역량인 성장역량을 50% 반영한다는 점을 고려한 지원이 필요함. 진로가 명확하지\n않거나 중간에 변경된 학생이라도 능동적이고 성실하게 학교생활을 한 학생이라면 지원을 고려해 볼 필요가 있음.\n다. 논술전형'),
 Document(metadata={'source': './data/ssen_single_test.pdf', 'file_path': './data/ssen_single_test.pdf', 'page': 5, 'total_pages': 8, 'CreationDate': "D:20240908161921+09'00'", 'Creator': 'PDF 2022 12.0.0.3970', 'ModDate': "D:20240908162013+09'00'", 'Producer': 'PDF 2022 12.0.0.3970'}, page_content='도표를 해석하는 문제가 매년 출제되고 있음. 인문사회Ⅱ에서

In [ ]:
# 5. LLM 객체 생성
# llm = ChatOpenAI()  # 아무 것도 지정하지 않으면 기본값으로는 gpt-3.5-turbo 모델이 사용됨.
llm = ChatOpenAI(
    model="gpt-4o-mini",   # 이렇게 지정하면 지정한 모델을 사용해 응답을 받을 수 있음.
    temperature=0.56,  # 모델의 창의성 정도
    # max_tokens=None,
    # timeout=None,
    # max_retries=2,
)

# 6. Retriever 객체 생성
# [참고] Retriever는 일종의 인터페이스로 다양한 유형의 데이터 소스를 포함할 수 있음.
# Vector Store 역시 Retriever 객체가 될 수 있음.
# [중요] Retriever는 항상 입력으로 하나의 string을 받고, 출력으로는 List of Documents를 리턴함!
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# 7. ChatPromptTemplate 객체 생성
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        너는 한국의 대학 입시와 진학에 대한 전문가야. 주어진 문서 내용만 참고해서 질문에 해당하는 최종 답변을 작성해 줘. 절대 주어진 문서에 없는 내용으로 답을 지어내려고 노력하지 마. 주어진 문서에서 답을 찾을 수 없으면 그냥 찾을 수 없다고 말해. 답변은 항상 존대말로 해 줘.
        -----
        {context}
        """
    ),
    ("human", "{question}")
])

# LCEL 사용해 chain 객체 생성
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

In [ ]:
# chain 실행
chain.invoke("KU논술전형 지원전략 알려줘.")

AIMessage(content='KU논술전형의 지원전략으로는 다음과 같은 사항을 고려할 수 있습니다. 많은 학과의 논술전형 모집인원이 감소하였고, 줄어든 모집인원은 단과대학 자유전공학부와 KU자유전공학부에서 선발하고 있습니다. 단과대학 자유전공학부의 경우 기존의 논술 출제 범위와 같지만, KU자유전공학부의 경우 국어, 통합사회, 수학, 수학Ⅰ, 수학Ⅱ가 포함되어 있습니다. 모집단위의 신설, 통합, 모집인원 변동 등 변경 사항이 많아 경쟁률 추이를 지켜보며 신중하게 지원해야 합니다. 또한, 7월 8일 경 입학처 홈페이지에 공개되는 모의논술을 통해 문제 유형을 파악하는 것이 필요합니다.', response_metadata={'token_usage': {'completion_tokens': 189, 'prompt_tokens': 1446, 'total_tokens': 1635, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-55bbd83e-44ec-4d9e-b845-f9f9a8579482-0')

# Markdown 형식으로 변환된 파일을 사용하는 경우
```
- [중요] UnstructuredMarkdownLoader를 이용하면 Heading 마크업과 <page> 태그와 같은 것이 모두 제거됨.
- 따라서, UnstructuredMarkdownLoader 대신 TextLoader를 사용해서 로드해야 함!
- 그런데, TextLoader를 사용하면 문서에 포함된 Heading 마크업 정보를 LLM이 제거하지 않고 해당 마크업 정보를 그대로 살려서 답변하는 경우가 있음. (프롬프트로 잘 조절하거나 원본 문서에서 context를 위한 Heading 마크업은 XML 형식으로 삽입시키는 것도 좋을 것으로 생각됨)
```

In [ ]:
# 파일 포맷에 따라 Loader를 선택해 import하기
# from langchain.document_loaders import TextLoader, CSVLoader, PyPDFLoader
# from langchain_community.document_loaders import PyPDFLoader
# from langchain_community.document_loaders import PDFPlumberLoader
# from langchain_community.document_loaders import PDFMinerLoader
# from langchain_community.document_loaders import UnstructuredMarkdownLoader
# from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Vector Store 종류는 엄청 다양함. 이번에는 Chroma 대신 FAISS 사용
# from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

FILE_PATH = "./data/ssen_double_test.md"

# 1. 데이터 로드
# loader = UnstructuredMarkdownLoader(FILE_PATH)
# [중요] UnstructuredMarkdownLoader를 이용하면 Heading 마크업과 <page> 태그와 같은 것이 모두 제거됨.
# 따라서, TextLoader를 사용해서 로드해야 함!
loader = TextLoader(FILE_PATH, encoding='utf-8')
docs = loader.load()

In [ ]:
docs

[Document(metadata={'source': './data/ssen_double_test.md'}, page_content="# 건국대학교\n<page>17</page>\n- (05029) 서울특별시 광진구 능동로 120\n- 입학처: (02) 450-0007\n\n\n@@@@\n# 건국대학교\n## 2024 대입과 달라진 점\n<page>17</page>\n(표 생략)\n\n@@@@\n# 건국대학교\n## 전형 일정\n<page>17</page>\n(표 생략)\n\n\n@@@@\n# 건국대학교\n## 모집 단위 및 전형별 모집인원\n<page>18</page>\n(표 생략)\n<note>※ 모집단위 변경이 많으므로 '8. 2024, 2023 대입 수시모집 경쟁률' 하단의 모집단위 변경 참조</note>\n\n\n@@@@\n# 건국대학교\n## 전형 요소별 반영 비율\n<page>19</page>\n(표 생략)\n\n\n@@@@\n# 건국대학교\n## 수능최저학력기준\n<page>20</page>\n| 전형명 | 수능최저학력기준 |\n|---|---|\n| KU논술우수자 |  |\n| - 인문/자연 | 국,수,영,탐(1) 중 2개 합 5 이내, 한국사 5 이내 |\n| - KU자유전공학부 |  |\n| - 수의예과 | 국,수,영,탐(1) 중 3개 합 4 이내, 한국사 5 이내 |\n\n\n@@@@\n# 건국대학교\n## 학교생활기록부 반영 방법\n<page>20</page>\n### 전형별 교과영역 반영요소 및 반영교과(군)\n\n| 전형명 | 활용지표 | 반영교과(군) |\n|---|---|---|\n| KU지역균형 | 석차등급, 이수단위 | 인문, 자연: 국어, 수학, 영어, 사회, 과학, 한국사 전 과목 |\n| 사회통합 |  | 석차등급 및 이수단위가 표기된 전 과목 |\n|  |  | 학년별, 교과별 가중치 없음 |\n\n■ 교과성적 산출 방법: 전체 모집단위에 석차등급별 기준점수 적용\n\n| 석차등급 | 1 | 2 | 3 | 4 | 5 |

In [ ]:
docs[0]

Document(metadata={'source': './data/ssen_double_test.md'}, page_content="# 건국대학교\n<page>17</page>\n- (05029) 서울특별시 광진구 능동로 120\n- 입학처: (02) 450-0007\n\n\n@@@@\n# 건국대학교\n## 2024 대입과 달라진 점\n<page>17</page>\n(표 생략)\n\n@@@@\n# 건국대학교\n## 전형 일정\n<page>17</page>\n(표 생략)\n\n\n@@@@\n# 건국대학교\n## 모집 단위 및 전형별 모집인원\n<page>18</page>\n(표 생략)\n<note>※ 모집단위 변경이 많으므로 '8. 2024, 2023 대입 수시모집 경쟁률' 하단의 모집단위 변경 참조</note>\n\n\n@@@@\n# 건국대학교\n## 전형 요소별 반영 비율\n<page>19</page>\n(표 생략)\n\n\n@@@@\n# 건국대학교\n## 수능최저학력기준\n<page>20</page>\n| 전형명 | 수능최저학력기준 |\n|---|---|\n| KU논술우수자 |  |\n| - 인문/자연 | 국,수,영,탐(1) 중 2개 합 5 이내, 한국사 5 이내 |\n| - KU자유전공학부 |  |\n| - 수의예과 | 국,수,영,탐(1) 중 3개 합 4 이내, 한국사 5 이내 |\n\n\n@@@@\n# 건국대학교\n## 학교생활기록부 반영 방법\n<page>20</page>\n### 전형별 교과영역 반영요소 및 반영교과(군)\n\n| 전형명 | 활용지표 | 반영교과(군) |\n|---|---|---|\n| KU지역균형 | 석차등급, 이수단위 | 인문, 자연: 국어, 수학, 영어, 사회, 과학, 한국사 전 과목 |\n| 사회통합 |  | 석차등급 및 이수단위가 표기된 전 과목 |\n|  |  | 학년별, 교과별 가중치 없음 |\n\n■ 교과성적 산출 방법: 전체 모집단위에 석차등급별 기준점수 적용\n\n| 석차등급 | 1 | 2 | 3 | 4 | 5 | 

In [ ]:
# 2. OpenAI의 Tokenizer인 tiktoken 사용해 분할하기

# splitter = RecursiveCharacterTextSplitter(
#     # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
#     chunk_size=250,
#     # 청크 간의 중복되는 문자 수를 설정합니다.
#     chunk_overlap=0,
#     # 문자열 길이를 계산하는 함수를 지정합니다.
#     length_function=len,
#     # 구분자로 정규식을 사용할지 여부를 설정합니다.
#     is_separator_regex=False,
# )

# [중요] chunk_size에 구애받지 않고 무조건 separator 기준으로 분할하려면 chunk_size를 매우 적게 할당하면 됨!
# 그리고, 이렇게 하려는 경우에는 RecursiveCharacterTextSplitter를 사용하면 안되고 CharacterTextSplitter를 사용해야 함!!
splitter = CharacterTextSplitter(
    separator="@@@@",
    # chunk_size=30000,
    # chunk_overlap=6000,
    chunk_size=250,   # 매우 적은 숫자를 지정해 무조건 separator 기준으로만 분할하도록 함.
    chunk_overlap=0,
)

splitted_docs = splitter.split_documents(docs)

Created a chunk of size 456, which is longer than the specified 250
Created a chunk of size 1007, which is longer than the specified 250
Created a chunk of size 1965, which is longer than the specified 250
Created a chunk of size 2283, which is longer than the specified 250
Created a chunk of size 942, which is longer than the specified 250
Created a chunk of size 1222, which is longer than the specified 250
Created a chunk of size 1903, which is longer than the specified 250
Created a chunk of size 1613, which is longer than the specified 250


In [ ]:
splitted_docs

[Document(metadata={'source': './data/ssen_double_test.md'}, page_content='# 건국대학교\n<page>17</page>\n- (05029) 서울특별시 광진구 능동로 120\n- 입학처: (02) 450-0007\n\n\n@@@@\n# 건국대학교\n## 2024 대입과 달라진 점\n<page>17</page>\n(표 생략)\n\n@@@@\n# 건국대학교\n## 전형 일정\n<page>17</page>\n(표 생략)'),
 Document(metadata={'source': './data/ssen_double_test.md'}, page_content="# 건국대학교\n## 모집 단위 및 전형별 모집인원\n<page>18</page>\n(표 생략)\n<note>※ 모집단위 변경이 많으므로 '8. 2024, 2023 대입 수시모집 경쟁률' 하단의 모집단위 변경 참조</note>\n\n\n@@@@\n# 건국대학교\n## 전형 요소별 반영 비율\n<page>19</page>\n(표 생략)"),
 Document(metadata={'source': './data/ssen_double_test.md'}, page_content='# 건국대학교\n## 수능최저학력기준\n<page>20</page>\n| 전형명 | 수능최저학력기준 |\n|---|---|\n| KU논술우수자 |  |\n| - 인문/자연 | 국,수,영,탐(1) 중 2개 합 5 이내, 한국사 5 이내 |\n| - KU자유전공학부 |  |\n| - 수의예과 | 국,수,영,탐(1) 중 3개 합 4 이내, 한국사 5 이내 |'),
 Document(metadata={'source': './data/ssen_double_test.md'}, page_content='# 건국대학교\n## 학교생활기록부 반영 방법\n<page>20</page>\n### 전형별 교과영역 반영요소 및 반영교과(군)\n\n| 전형명 | 활용지표 | 반영교과(군) |\n|---|---|---

In [ ]:
# 3. OpenAI 임베딩 객체 생성
embeddings = OpenAIEmbeddings()
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")   # 최신 임베딩 모델로 지정! (저렴하고 성능도 우수함) --> 모델을 못 찾음. ㅠ

# 4. Vector Store 생성 및 캐싱
# [주의] OpenAIEmbeddings() 객체인 embeddings를 사용해서 분할한 문서에 대한 vectorization을 진행하기 때문에 Embedding 모델 사용료가 나감!! (따라서, 여러번 실행하지 말 것!!)
cache_dir = LocalFileStore("./.cache/jinhak/")
# 첫번째 인자로 캐싱할 OpenAIEmbeddings 객체를 전달
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 캐싱 디렉토리에 캐싱된 임베딩이 있으면 그것을 사용해서 Vector Store 생성
# 만야, 맨 처음 실행되는 상황이라 캐싱된 임베딩이 없으면 cache_embeddings 생성할 때 인자로 넘겨준 최초의 임베딩을 사용함.
# vectorstore = Chroma.from_documents(splitted_docs4, cache_embeddings)
# 실제로 해 보면 Chroma에 비해 FAISS를 사용했을 때 더 좋은 성능을 보이는 경우가 많다고 함.
vectorstore = FAISS.from_documents(splitted_docs, cache_embeddings)

In [ ]:
# [테스트] Vector Store에서 인자로 넘긴 문장과 유사도가 높은 청크 문서 찾기
result_docs = vectorstore.similarity_search("건국대학교 전형 종류에 대해 알려줘.")
# result_docs = vectorstore.similarity_search("전형 종류에 대해 알려줘.", k=6)  # 반환되는 문서 수를 6개로 설정(기본값: 4개)
# result_docs = vectorstore.similarity_search("서강대학교 입시 결과 알려줘.")
result_docs

[Document(metadata={'source': './data/ssen_double_test.md'}, page_content='# 건국대학교\n<page>17</page>\n- (05029) 서울특별시 광진구 능동로 120\n- 입학처: (02) 450-0007\n\n\n@@@@\n# 건국대학교\n## 2024 대입과 달라진 점\n<page>17</page>\n(표 생략)\n\n@@@@\n# 건국대학교\n## 전형 일정\n<page>17</page>\n(표 생략)'),
 Document(metadata={'source': './data/ssen_double_test.md'}, page_content='# 서강대학교\n## 주요 전형 분석 및 지원 전략\n### 논술전형\n<page>107</page>\n| 전형명 | 모집인원(명) | 전형방법 | 수능최저학력기준 |\n|:---:|:---:|:---:|:---:|\n| 일반 | 173 | 논술 80+교과 10+비교과(출결) 10 | 적용 |\n\n#### 출제 유형 및 평가방법\n- 고사계열\n  | 고사계열 | 출제분야 | 수학 출제 범위 |\n  |:---:|:---:|:---:|\n  | 인문, 인문·자연 | 인문/사회과학 관련 제시문과 논제 |  |\n  | 자연 | 수리 관련 제시문과 논제 | 수학, 수학, 수학II, 미적분, 확률과 통계, 기하 |\n- 답안 작성 분량\n  | 고사계열 | 문제1 | 문제2 | 답안 작성 분량 | 시험시간 |\n  |:---:|:---:|:---:|:---:|:---:|\n  | 자연 | 40 | 60 | 1문제당 800 ~ 1,000자 분량 제한 없음(문제당 1쪽 이내) | 100분 |\n\n#### 지원전략\n• 2023 대입과 2024 대입의 논술전형을 비교해 보면, 같은 인원인 175명을 모집하는데 전체 경쟁률은 각각 94.6:1에서 112.6:1로 높아졌으며 지원자는 3,152명이 증가함. 수능최저학력기준은 2023 대입 국수영탐(1) 4개 영역 중 

In [ ]:
# 5. LLM 객체 생성
# llm = ChatOpenAI()  # 아무 것도 지정하지 않으면 기본값으로는 gpt-3.5-turbo 모델이 사용됨.
llm = ChatOpenAI(
    model="gpt-4o-mini",   # 이렇게 지정하면 지정한 모델을 사용해 응답을 받을 수 있음.
    temperature=0.1,  # 모델의 창의성 정도
    # max_tokens=None,
    # timeout=None,
    # max_retries=2,
)

# 6. Retriever 객체 생성
# [참고] Retriever는 일종의 인터페이스로 다양한 유형의 데이터 소스를 포함할 수 있음.
# Vector Store 역시 Retriever 객체가 될 수 있음.
# [중요] Retriever는 항상 입력으로 하나의 string을 받고, 출력으로는 List of Documents를 리턴함!
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# 7. ChatPromptTemplate 객체 생성
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        너는 한국의 대학 입시와 진학 지도에 대한 전문가야. 주어진 문서 내용만 참고해서 질문에 해당하는 최종 답변을 작성해 줘. 절대 주어진 문서에 없는 내용으로 답을 지어내려고 노력하지 마. 주어진 문서에서 답을 찾을 수 없으면 그냥 찾을 수 없다고 말해. 답변은 항상 존대말로 해 줘.

        - 답변의 마지막에는 blank line을 하나 추가한 다음 너가 답변을 생성하는데 있어서 주로 참고한 문서의 page 번호를 '[출처] 쎈(sen)진학 2024 수시 대학별 분석, ' 문자열과 concate해서 함께 답변해 줘. page 번호는 너가 참고한 문서 내의 <page> element 내용을 참고해. (예시: page 번호가 5인 경우라면, '[출처] 쎈(sen)진학 2024 수시 대학별 분석, 5페이지')
        - 주어진 문서에서 답을 찾을 수 없거나 답변할 내용이 거의 없는 경우에는 출처 정보를 무조건 생략하고 답변해.
        - '지원전략'에 대한 질문에 답변할 때는 문서에 포함된 내용을 생략하거나 요약, 변경하지 말고 원본 문서에 포함된 '지원전략' 내용을 그대로 답변해 줘.
        - 문서에 포함된 내용이 아닌 너의 opinion(의견)에 대해 질문받는 경우에는, 먼저 문서에 포함된 내용을 답변하고 마지막에 너의 opinion은 '(의견) '이라는 머리글로 시작해서 답변해 줘.
        -----
        {context}
        """
    ),
    ("human", "{question}")
])

# LCEL 사용해 chain 객체 생성
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

## 질문-답변 테스트

In [ ]:
# chain 실행
chain.invoke("서강대 학생부종합전형 지원전략 알려줘.")

AIMessage(content="서강대학교 학생부종합전형의 지원전략은 다음과 같습니다.\n\n2023 대입과 2024 대입의 합격자 성적(70%컷)을 비교해 보면 인문계열(지식융합미디어학부 포함) 모집단위는 2.74 등급에서 2.91등급으로, 자연계열 모집단위는 2.39등급에서 2.75등급으로 다소 낮아짐. 합격자 성적의 변동 폭이 큰 모집단위의 경우 인문계열은 심리학과(2.71 → 1.74), 글로벌한국학부(1.83 → 3.43), 중국문화학과(2.88→3.39)였으며, 자연계열은 물리학과(2.22 → 4.43), 시스템반도체공학과(2.86 → 4.24), 생명과학과(1.74 → 2.87)였음. 최종등록자 70%컷은 합격 컷이 아니므로 지원 시 참고 바람. \n\n• 인문계열의 모집단위 중 사회학과, 정치외교학과, 심리학과의 합격자 성적(70%컷)은 각각 1.98, 1.80, 1.74등급으로 다른 모집단위에 비해 높은 편임. 이는 여러 다양한 유형의 학교가 지원하는 학생부종합전형의 특성을 고려해 보면 이 모집단위에서는 일반고 학생 지원이 더 많다는 것을 예상할 수 있음. \n\n• 2024 대입의 학생부종합전형에서 지원 및 등록 비율을 살펴보면, 지원자는 재학생과 졸업생 비율이 각각 59.6%, 40.4%였으며, 합격하여 등록한 학생은 각각 80.9%, 19.1%였음. 2023 대입에 비해 10% 이상 졸업생 지원이 늘었지만, 실제 합격한 등록생은 여전히 재학생이 많음. 졸업생의 경우 3학년 2학기까지 학교 생활을 충실히 했는가가 영향을 미칠 수 있다고 대학 측은 밝힘. \n\n• 평가 요소로 진로역량이나 전공적합성보다 넓은 의미인 '성장가능성'을 사용함. 이는 계열, 전공, 인원, 학점과 관계없이 원하는 전공을 이수할 수 있는 다전공제도를 운영하는 서강대의 특성을 반영하는 요소임. 지원자가 특정 전공과 관련된 활동을 했는지보다는 본인의 전공과 진로를 탐색해 나가는 과정을 통해 대학 입학 후에도 발전할 가능성이 있는지에 방점을 두고 평가함. \n\n• 교과

In [ ]:
# chain 실행
chain.invoke("건국대와 서강대의 수능최저조건을 비교해줘.")

AIMessage(content='건국대학교와 서강대학교의 수능최저학력기준은 다음과 같습니다.\n\n**건국대학교:**\n- KU논술우수자 전형: \n  - 인문/자연: 국, 수, 영, 탐(1) 중 2개 합 5 이내, 한국사 5 이내\n  - KU자유전공학부: 수능최저학력기준 없음\n  - 수의예과: 국, 수, 영, 탐(1) 중 3개 합 4 이내, 한국사 5 이내\n\n**서강대학교:**\n- 학생부교과(지역균형) 전형: 국, 수, 영, 탐(1) 중 3개 각 3등급, 한국사 4 이내\n- 논술(일반) 전형: 국, 수, 영, 탐(1) 중 3개 합 7 이내, 한국사 4 이내\n\n비교해보면, 건국대학교는 특정 전형에 따라 수능최저학력기준이 다르며, 수의예과의 경우 3개 과목 합 4 이내의 기준이 있습니다. 반면 서강대학교는 학생부교과전형과 논술전형 모두에서 3개 과목에 대한 기준을 두고 있으며, 학생부교과전형은 각 과목 3등급, 논술전형은 3개 과목 합 7 이내로 설정되어 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 326, 'prompt_tokens': 1505, 'total_tokens': 1831, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-565cebf8-89d8-4d76-a83d-6aa898b2176d-0', usage_metadata={'input_tokens': 1505, 'output_tokens': 326, 'total_tokens': 1831})

In [ ]:
# chain 실행
chain.invoke("건국대와 서강대의 수능최저조건을 정리해 주고, 너의 의견도 함께 알려줘.")

AIMessage(content='건국대학교와 서강대학교의 수능최저학력기준은 다음과 같습니다.\n\n**건국대학교**\n- KU논술우수자 전형: \n  - 인문/자연: 국, 수, 영, 탐(1) 중 2개 합 5 이내, 한국사 5 이내\n- KU자유전공학부: 수능최저학력기준 없음\n- 수의예과: 국, 수, 영, 탐(1) 중 3개 합 4 이내, 한국사 5 이내\n\n**서강대학교**\n- 학생부교과(지역균형): 국, 수, 영, 탐(1) 중 3개 각 3등급, 한국사 4 이내\n- 논술(일반): 국, 수, 영, 탐(1) 중 3개 합 7 이내, 한국사 4 이내\n\n[의견] 두 대학의 수능최저학력기준을 비교해보면, 건국대학교는 특정 전형에서 수능최저학력기준이 상대적으로 유연한 반면, 서강대학교는 학생부교과전형에서 각 과목의 등급을 요구하는 등 더 엄격한 기준을 가지고 있습니다. 따라서 지원자는 자신의 성적과 전형의 특성을 잘 고려하여 준비하는 것이 중요하다고 생각합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 300, 'prompt_tokens': 1569, 'total_tokens': 1869, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-c09013c7-f5f6-40d0-83e3-e378bf597cf0-0', usage_metadata={'input_tokens': 1569, 'output_tokens': 300, 'total_tokens': 1869})

In [ ]:
# chain 실행
chain.invoke("건국대와 서강대의 수능최저조건을 정리해 줘.")

AIMessage(content='건국대학교와 서강대학교의 수능최저학력기준은 다음과 같습니다.\n\n**건국대학교**\n- KU논술우수자 전형\n  - 인문/자연: 국, 수, 영, 탐(1) 중 2개 합 5 이내, 한국사 5 이내\n- KU자유전공학부 전형: 수능최저학력기준 없음\n- 수의예과 전형: 국, 수, 영, 탐(1) 중 3개 합 4 이내, 한국사 5 이내\n\n**서강대학교**\n- 학생부교과(지역균형) 전형: 국, 수, 영, 탐(1) 중 3개 각 3등급, 한국사 4 이내\n- 논술(일반) 전형: 국, 수, 영, 탐(1) 중 3개 합 7 이내, 한국사 4 이내\n\n[출처] 건국대학교: page 20, 서강대학교: page 105', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 228, 'prompt_tokens': 963, 'total_tokens': 1191, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-ef977778-e0e0-4c90-88dd-efc0b48b3dbb-0', usage_metadata={'input_tokens': 963, 'output_tokens': 228, 'total_tokens': 1191})

In [ ]:
# chain 실행
chain.invoke("서강대 비교과 반영 방법은?")

AIMessage(content='서강대학교의 비교과(출결) 반영 방법은 다음과 같습니다:\n\n- 전형명: 학생부교과(지역균형), 논술(일반)\n- 반영영역: 출결\n- 배점: 100점\n- 반영교과(군): \n  - 미인정 결석일 반영 점수\n    - 0~3일: 100점\n    - 4~6일: 98점\n    - 7~9일: 96점\n    - 10~14일: 90점\n    - 15일 이상: 0점\n- 총 결석 일수 = 결석 일수 + (지각 일수 + 조퇴 일수 + 결과 일수) / 3\n- 총 결석일수는 소수점 첫째 자리에서 버림하여 정수로 최종 산출합니다.\n\n[출처] 쎈(sen)진학 2024 수시 대학별 분석, 106페이지', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 217, 'prompt_tokens': 2766, 'total_tokens': 2983, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-2334f288-98a9-4c57-9552-3f685eb2204b-0', usage_metadata={'input_tokens': 2766, 'output_tokens': 217, 'total_tokens': 2983})

In [ ]:
# chain 실행
chain.invoke("서강대 전형 종류와 각 전형별 기본 정보 알려줘.")

AIMessage(content='서강대학교의 전형 종류와 각 전형별 기본 정보는 다음과 같습니다.\n\n1. **학생부종합전형**\n   - 모집인원: 558명\n   - 전형방법: 서류 100%\n   - 수능최저학력기준: 미적용\n\n2. **논술전형**\n   - 모집인원: 173명\n   - 전형방법: 논술 80% + 교과 10% + 비교과(출결) 10%\n   - 수능최저학력기준: 적용\n\n3. **학생부교과전형 (지역균형)**\n   - 모집인원: 178명\n   - 전형방법: 교과 90% + 비교과(출결) 10%\n   - 수능최저학력기준: 적용\n\n각 전형의 세부 사항은 위와 같습니다. \n\n  \n\n[출처] 쎈(sen)진학 2024 수시 대학별 분석, 105페이지', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 220, 'prompt_tokens': 3650, 'total_tokens': 3870, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-b591cc0f-419a-4537-81d8-d888e35b66b5-0', usage_metadata={'input_tokens': 3650, 'output_tokens': 220, 'total_tokens': 3870})

In [ ]:
# chain 실행
chain.invoke("서강대 전형 종류에 대해 자세히 알려줘.")

AIMessage(content='서강대학교의 전형 종류는 다음과 같습니다.\n\n1. **학생부종합전형**\n   - 모집인원: 558명\n   - 전형방법: 서류 100%\n   - 수능최저학력기준: 미적용\n\n2. **논술전형**\n   - 모집인원: 173명\n   - 전형방법: 논술 80% + 교과 10% + 비교과(출결) 10%\n   - 수능최저학력기준: 적용\n\n3. **학생부교과전형**\n   - 모집인원: 178명\n   - 전형방법: 교과 90% + 비교과(출결) 10%\n   - 수능최저학력기준: 적용\n\n각 전형은 평가 방법과 지원 전략이 다르므로, 지원하고자 하는 전형에 대한 충분한 이해가 필요합니다. \n\n  \n\n[출처] 쎈(sen)진학 2024 수시 대학별 분석, 106페이지', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 225, 'prompt_tokens': 3570, 'total_tokens': 3795, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-7dd8dd97-4146-4022-b8d6-5231b5f49ff5-0', usage_metadata={'input_tokens': 3570, 'output_tokens': 225, 'total_tokens': 3795})

In [ ]:
# chain 실행
chain.invoke("서강대 학생부종합전형 이름이 뭐야?")

AIMessage(content="서강대학교의 학생부종합전형 이름은 '일반'입니다. \n\n  \n[출처] 쎈(sen)진학 2024 수시 대학별 분석, 107페이지", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 1976, 'total_tokens': 2020, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-2680ad8f-a19f-48c1-a5a4-e318998a900a-0', usage_metadata={'input_tokens': 1976, 'output_tokens': 44, 'total_tokens': 2020})

In [ ]:
# chain 실행
chain.invoke("서강대 학생부 반영 방법 알려줘.")

AIMessage(content='서강대학교의 학교생활기록부 반영 방법은 다음과 같습니다.\n\n### 전형별 교과영역 반영요소 및 반영교과(군)\n- **학생부교과(지역균형)**: 석차등급, 이수단위, 성취도 및 성취비율\n  - 반영교과: 석차등급, 이수단위, 성취도 및 성취비율이 부여되는 전 과목, 학년별, 교과별 가중치 없음\n- **논술(일반)**: 석차등급, 이수단위\n  - 반영교과: 석차등급이 부여되는 전 과목, 학년별, 교과별 가중치 없음\n\n### 교과성적 산출 방법: 등급평균 산출 후 최종점수 부여\n- **석차등급**: \n  1) 등급평균 = ∑(반영 과목 등급 × 단위수) / ∑(반영 과목 단위수) (9등급)\n- **등급계산**: \n  2) 최종점수 = (9 - 등급평균) × 100\n\n### 진로선택과목 및 전문교과 성적 반영 방법: 정량평가\n- **비율계산**: \n  1) 과목 환산 성취 비율 = 취득 성취 비율 /2 + 성취도 하단 성취비율 합계 \n  2) 최종점수 = (∑(반영 과목 환산성취비율)) / 2 \n  3) 최종점수가 100점 이상인 경우 모두 100점으로 처리\n\n### 비교과(출결) 반영 방법\n- **학생부교과(지역균형) 논술(일반)**: 출결\n  - 배점: 100점\n  - 반영교과: 미인정 결석일 반영 점수 (0~3일 100, 4~6일 98, 7~9일 96, 10~14일 90, 15일 이상 0)\n\n이와 같은 방법으로 서강대학교는 학생부를 반영하고 있습니다.\n\n  \n\n[출처] 쎈(sen)진학 2024 수시 대학별 분석, 106페이지', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 522, 'prompt_tokens': 2487, 'total_tokens': 3009, 'completion_tokens_details': {'reasoning_tokens': 0}}, 

In [ ]:
# chain 실행
chain.invoke("서강대 학생부종합 기본 정보 알려줘.")

AIMessage(content='서강대학교의 학생부종합전형에 대한 기본 정보는 다음과 같습니다.\n\n- 전형명: 일반\n- 모집인원: 558명\n- 전형방법: 서류 100%\n- 수능최저학력기준: 미적용\n\n평가 방법 및 내용은 서류평가로 진행되며, 평가요소는 학업역량, 공동체역량, 성장가능성으로 나뉘어 있습니다. 각 평가요소의 성취수준과 창의적 문제해결력에 대한 비율도 명시되어 있습니다.\n\n  \n\n[출처] 쎈(sen)진학 2024 수시 대학별 분석, 107페이지', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 1974, 'total_tokens': 2125, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-71372342-fd6c-4903-b5d9-07a96adbe658-0', usage_metadata={'input_tokens': 1974, 'output_tokens': 151, 'total_tokens': 2125})

In [ ]:
# chain 실행
chain.invoke("서강대 학생부교과 지원자격 알려줘.")

AIMessage(content='서강대학교 학생부교과전형의 지원자격은 다음과 같습니다:\n\n- 국내 고등학교 해당 학년도 졸업예정자 중 국내 고교에서 4학기 이상 성적을 취득한 자로서 출신 고등학교장의 추천을 받은 자\n- 고교별 최대 추천 가능 인원: 20명\n- 지원 불가: 마이스터고, 특성화고, 전문계 과정(일반고, 종합고), 예술고, 체육고, 학력인정 평생학교, 각종학교 및 대안학교, 방송통신고, 기타 학생부 성적체계가 다른 고교\n\n  \n\n[출처] 쎈(sen)진학 2024 수시 대학별 분석, 106페이지', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 2636, 'total_tokens': 2805, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-c2324031-845b-44bf-a4ab-eaffa42e11ad-0', usage_metadata={'input_tokens': 2636, 'output_tokens': 169, 'total_tokens': 2805})

In [ ]:
# chain 실행
chain.invoke("서강대 논술 지원자격 알려줘.")

AIMessage(content='서강대학교 논술전형의 지원자격에 대한 구체적인 내용은 문서에 포함되어 있지 않습니다. 따라서 지원자격에 대한 정보를 제공할 수 없습니다. \n\n', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 2966, 'total_tokens': 3006, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-f756a870-6e1d-4c14-b6fd-e73e5b7d5c63-0', usage_metadata={'input_tokens': 2966, 'output_tokens': 40, 'total_tokens': 3006})

### 답변을 찾을 수 없다고 답변해야 하는 경우
```
- 아예 내용이 없는 경우
- 소제목과는 일치하지만 구체적인 내용이 없는 경우
- 두 경우의 답변 내용과 출처 표시 여부를 비교해야 함!!
```

In [ ]:
# chain 실행
chain.invoke("서강대 전형일정 알려줘.")

AIMessage(content='서강대학교의 전형 일정에 대한 구체적인 내용은 문서에 포함되어 있지 않습니다. \n\n  \n\n[출처] 쎈(sen)진학 2024 수시 대학별 분석, 104페이지', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 892, 'total_tokens': 941, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e1f5333-5d56-427f-9bd5-a0da814399b7-0', usage_metadata={'input_tokens': 892, 'output_tokens': 49, 'total_tokens': 941})

In [ ]:
# chain 실행
chain.invoke("서울대 지원전략 알려줘.")

AIMessage(content='죄송하지만, 주어진 문서에는 서울대학교에 대한 지원전략 내용이 포함되어 있지 않습니다. 다른 질문이 있으시면 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 2615, 'total_tokens': 2648, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-4d46ba19-a808-4748-9c02-4b3c2b5b8507-0', usage_metadata={'input_tokens': 2615, 'output_tokens': 33, 'total_tokens': 2648})

### 잘못된 답변을 하는 경우

In [ ]:
# chain 실행
chain.invoke("서강대 학생부종합 지원자격 알려줘.")    # 잘못된 답변임!! 학생부교과전형만 지원자격 정보가 있음.

AIMessage(content='서강대학교 학생부종합전형의 지원자격은 다음과 같습니다.\n\n• 국내 고등학교 해당 학년도 졸업예정자 중 국내 고교에서 4학기 이상 성적을 취득한 자로서 출신 고등학교장의 추천을 받은 자  \n• 고교별 최대 추천 가능 인원: 20명  \n• 지원 불가: 마이스터고, 특성화고, 전문계 과정(일반고, 종합고), 예술고, 체육고, 학력인정 평생학교, 각종학교 및 대안학교, 방송통신고, 기타 학생부 성적체계가 다른 고교  \n\n  \n\n[출처] 쎈(sen)진학 2024 수시 대학별 분석, 107페이지', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 2687, 'total_tokens': 2856, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac153aec-6ddb-4ca0-ad18-05c8990a3511-0', usage_metadata={'input_tokens': 2687, 'output_tokens': 169, 'total_tokens': 2856})